In [ ]:
#GCP Auth
# project_id = 'cloud-demo-393319'
project_id = 'cap5930-project-1'

# # this will launch an interactive prompt that allows you to auth with GCP in a browser
!gcloud auth application-default login --no-launch-browser

# # this will set your active project to the `project_id` above
!gcloud auth application-default set-quota-project $project_id

# print()

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=hhGHGOVtXD7DsIhSNXDjOCNu5mf5QI&prompt=consent&token_usage=remote&access_type=offline&code_challenge=Zdd_WjtwlyB-C9HhYLx2_2m1_PEOZFpfmoF2hnl53_s&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AQlEd8zd13yYCsQ3y1tsXWfxonsi3aPduAkPgM4wQEWUnYrHkYeZ3noRg9mMbUp6OthiSg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
# Authenticate your user/environemnt in colab to your project in google cloud
from google.colab import auth
auth.authenticate_user(project_id=project_id)

In [ ]:
!python3 -m pip install google-cloud-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.4/292.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
from google.cloud import speech
from google.protobuf import wrappers_pb2

client=speech.SpeechClient()

def sample_recognize(content):
  audio=speech.RecognitionAudio(content=content)

  config=speech.RecognitionConfig(
  # encoding=speech.RecognitionConfig.AudioEncoding.MP3,
  # sample_rate_hertz=24000,
  language_code="en-US",
  model="latest_long",
  audio_channel_count=2,
  enable_word_confidence=True,
  enable_word_time_offsets=True,
  )

  operation=client.long_running_recognize(config=config, audio=audio)

  response=operation.result(timeout=90)

  txt = ''
  for result in response.results:
    txt = txt + result.alternatives[0].transcript + '\n'

  return txt


In [ ]:
filename = 'ssml.wav'

f = open(filename,'rb')
data = f.read()
f.close()

text = sample_recognize(data)

print(text)

f = open(filename+'.txt','w')
f.write(text)
f.close()

one two three one two three
 repeating again one two three one two three



In [ ]:
!python3 -m pip install google-cloud-texttospeech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
from google.cloud import texttospeech_v1

client = texttospeech_v1.TextToSpeechClient()

#####
## Print available voices
#####
request = texttospeech_v1.ListVoicesRequest()
voices = client.list_voices(request=request)
print(voices)
#####

voices {
  language_codes: "af-ZA"
  name: "af-ZA-Standard-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "am-ET"
  name: "am-ET-Standard-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "am-ET"
  name: "am-ET-Standard-B"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "am-ET"
  name: "am-ET-Wavenet-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "am-ET"
  name: "am-ET-Wavenet-B"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name: "ar-XA-Standard-A"
  ssml_gender: FEMALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name: "ar-XA-Standard-B"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name: "ar-XA-Standard-C"
  ssml_gender: MALE
  natural_sample_rate_hertz: 24000
}
voices {
  language_codes: "ar-XA"
  name:

In [ ]:


def sample_synthesize_speech(text=None, ssml=None):
    input = texttospeech_v1.SynthesisInput()
    if ssml:
      input.ssml = ssml
    else:
      input.text = text

    voice = texttospeech_v1.VoiceSelectionParams()
    voice.language_code = "en-US"
    # voice.ssml_gender = "MALE"

    audio_config = texttospeech_v1.AudioConfig()
    audio_config.audio_encoding = "LINEAR16"

    request = texttospeech_v1.SynthesizeSpeechRequest(
        input=input,
        voice=voice,
        audio_config=audio_config,
    )

    response = client.synthesize_speech(request=request)

    return response.audio_content

In [ ]:
text = "1234567890"

wav = sample_synthesize_speech(text)

# save audio
f = open('tts.wav','wb')
f.write(wav)
f.close()

#save text
f = open('tts.wav.txt','w')
f.write(text)
f.close()

In [ ]:
from IPython.display import Audio, display

display(Audio('tts.wav', autoplay=True))

In [ ]:
ssml = """<speak>
            <say-as interpret-as='characters'>
              1234567890
            </say-as>
          </speak>"""

wav = sample_synthesize_speech(text=None, ssml=ssml)

f = open('ssml.wav','wb')
f.write(wav)
f.close()

In [ ]:
from IPython.display import Audio, display

display(Audio('ssml.wav', autoplay=True))